### Make Backtest

In [15]:
import pandas as pd
import sys
sys.path.append('../../')
import helpers.backtest as bt_helper

name = "some_bt"
year = 2024
sample_size = 0
start, end = '2024-09-10', '2024-09-11'
start_time, end_time = "13:00:00", "13:30:00"
instruments = ["MES"]
pvars = [
    {
        "name": "account_value",
        "type": "vector",
        "study_name":
        "Account Balance Graph",
        "pkey": 2
    },
]
sg_arrays = []

_ = bt_helper.generate_backtest(
    name,
    year,
    instruments,
    sample_size,
    start, end,
    start_time, end_time,
    pvars, sg_arrays,
)

n days -> 2
contracts -> {'MES-202409-CME-USD'}
copy depth files..
current instrument -> MES
 100.00% 2/2... rate=0.13 Hz, eta=0:00:00, total=0:00:15
done.


0